![piggy_bank](piggy_bank.jpg)

<br>

Personal loans are a lucrative revenue stream for banks. The typical interest rate of a two year loan in the United Kingdom is [around 10%](https://www.experian.com/blogs/ask-experian/whats-a-good-interest-rate-for-a-personal-loan/). This might not sound like a lot, but in September 2022 alone UK consumers borrowed [around £1.5 billion](https://www.ukfinance.org.uk/system/files/2022-12/Household%20Finance%20Review%202022%20Q3-%20Final.pdf), which would mean approximately £300 million in interest generated by banks over two years!

You have been asked to work with a bank to clean and store the data they collected as part of a recent marketing campaign, which aimed to get customers to take out a personal loan. They plan to conduct more marketing campaigns going forward so would like you to set up a PostgreSQL database to store this campaign's data, designing the schema in a way that would allow data from future campaigns to be easily imported. 

They have supplied you with a csv file called `"bank_marketing.csv"`, which you will need to clean, reformat, and split, in order to save separate files based on the tables you will create. It is recommended to use `pandas` for these tasks.

Lastly, you will write the SQL code that the bank can execute to create the tables and populate with the data from the csv files. As the bank are quite strict about their security, you'll provide the database design script as a `.sql` file that they can then run. 

You have been asked to design a database that will have three tables:

## client

| column | data type | description |
|--------|-----------|-------------|
| `id` | `serial` | Client ID - primary key |
| `age` | `integer` | Client's age in years |
| `job` | `text` | Client's type of job |
| `marital` | `text` | Client's marital status |
| `education` | `text` | Client's level of education |
| `credit_default` | `boolean` | Whether the client's credit is in default |
| `housing` | `boolean` | Whether the client has an existing housing loan (mortgage) |
| `loan` | `boolean` | Whether the client has an existing personal loan |

<br>

## campaign

| column | data type | description |
|--------|-----------|-------------|
| `campaign_id` | `serial` | Campaign ID - primary key |
| `client_id` | `serial` | Client ID - references `id` in the `client` table |
| `number_contacts` | `integer` | Number of contact attempts to the client in the current campaign |
| `contact_duration` | `integer` | Last contact duration in seconds |
| `pdays` | `integer` | Number of days since contact in previous campaign (`999` = not previously contacted) |
| `previous_campaign_contacts` | `integer` | Number of contact attempts to the client in the previous campaign |
| `previous_outcome` | `boolean` | Outcome of the previous campaign |
| `campaign_outcome` | `boolean` | Outcome of the current campaign |
| `last_contact_date` | `date` | Last date the client was contacted |

<br>

## economics

| column | data type | description |
|--------|-----------|-------------|
| `client_id` | `serial` | Client ID - references `id` in the `client` table |
| `emp_var_rate` | `float` | Employment variation rate (quarterly indicator) |
| `cons_price_idx` | `float` | Consumer price index (monthly indicator) |
| `euribor_three_months` | `float` | Euro Interbank Offered Rate (euribor) three month rate (daily indicator) |
| `number_employed` | `float` | Number of employees (quarterly indicator)| 

In [51]:
import pandas as pd
import numpy as np

df = pd.read_csv('bank_marketing.csv')
print(df)

       client_id  age          job  ... euribor3m nr_employed    y
0              0   56    housemaid  ...     4.857      5191.0   no
1              1   57     services  ...     4.857      5191.0   no
2              2   37     services  ...     4.857      5191.0   no
3              3   40       admin.  ...     4.857      5191.0   no
4              4   56     services  ...     4.857      5191.0   no
...          ...  ...          ...  ...       ...         ...  ...
41183      41183   73      retired  ...     1.028      4963.6  yes
41184      41184   46  blue-collar  ...     1.028      4963.6   no
41185      41185   56      retired  ...     1.028      4963.6   no
41186      41186   44   technician  ...     1.028      4963.6  yes
41187      41187   74      retired  ...     1.028      4963.6   no

[41188 rows x 22 columns]


In [52]:
print(df.columns)

Index(['client_id', 'age', 'job', 'marital', 'education', 'credit_default',
       'housing', 'loan', 'contact', 'month', 'day', 'duration', 'campaign',
       'pdays', 'previous', 'poutcome', 'emp_var_rate', 'cons_price_idx',
       'cons_conf_idx', 'euribor3m', 'nr_employed', 'y'],
      dtype='object')


In [53]:
client = df[['client_id', 'age', 'job', 'marital', 'education', 'credit_default', 'housing', 'loan']]
campaign = df[["client_id", "campaign", "month", "day", 
               "duration", "pdays", "previous", "poutcome", "y"]]
economics = df[['emp_var_rate', 'cons_price_idx', 'cons_conf_idx', 'euribor3m', 'nr_employed']]

In [54]:
client = client.rename(columns={'client_id': 'id'})

campaign = campaign.rename(columns={'duration': 'contact_duration',
                                    'previous': 'previous_campaign_contacts',
                                    'y': 'campaign_outcome',
                                    'poutcome': 'previous_outcome',
                                    'campaign': 'number_contacts'})

economics = economics.rename(columns={'euribor3m': 'euribor_three_months',
                                      'nr_employed': 'number_employed'})

In [55]:
client['education'] = client['education'].str.replace('.','_')
client['job'] = client['job'].str.replace('.','')

In [56]:
client['education'] = client['education'].replace('unknown',np.nan)

In [57]:
client

,id,age,job,marital,education,credit_default,housing,loan
0,0,56,housemaid,married,basic_4y,no,no,no
1,1,57,services,married,high_school,unknown,no,no
2,2,37,services,married,high_school,no,yes,no
3,3,40,admin,married,basic_6y,no,no,no
4,4,56,services,married,high_school,no,no,yes
...,...,...,...,...,...,...,...,...
41183,41183,73,retired,married,professional_course,no,yes,no
41184,41184,46,blue-collar,married,professional_course,no,no,no
41185,41185,56,retired,married,university_degree,no,yes,no
41186,41186,44,technician,married,professional_course,no,no,no


In [58]:
campaign[campaign['previous_outcome'] == 'success']

,client_id,number_contacts,month,day,contact_duration,pdays,previous_campaign_contacts,previous_outcome,campaign_outcome
24108,24108,1,nov,25,119,6,1,success,no
24264,24264,1,nov,21,112,4,1,success,no
24279,24279,1,nov,28,94,4,1,success,no
24397,24397,1,nov,11,77,3,1,success,no
24482,24482,2,nov,11,200,4,1,success,no
...,...,...,...,...,...,...,...,...,...
41163,41163,3,nov,13,385,4,2,success,yes
41164,41164,2,nov,17,1868,10,1,success,yes
41174,41174,1,nov,14,208,1,6,success,yes
41178,41178,2,nov,26,483,6,3,success,yes


In [59]:
campaign['campaign_outcome'] = campaign['campaign_outcome'].replace({'yes': 1, 'no': 0})
campaign['previous_outcome'] = campaign['previous_outcome'].replace({'success': 1, 'failure': 0, 'nonexistent': np.nan})

In [60]:
campaign[campaign['campaign_outcome'] == 1]

,client_id,number_contacts,month,day,contact_duration,pdays,previous_campaign_contacts,previous_outcome,campaign_outcome
75,75,1,may,20,1575,999,0,NaN,1
83,83,1,may,6,1042,999,0,NaN,1
88,88,1,may,19,1467,999,0,NaN,1
129,129,1,may,12,579,999,0,NaN,1
139,139,1,may,27,461,999,0,NaN,1
...,...,...,...,...,...,...,...,...,...
41174,41174,1,nov,14,208,1,6,1.0,1
41178,41178,2,nov,26,483,6,3,1.0,1
41181,41181,1,nov,20,281,999,0,NaN,1
41183,41183,1,nov,30,334,999,0,NaN,1


In [61]:
campaign['campaign_id'] = 1
campaign["month"] = campaign["month"].str.capitalize()
campaign["year"] = "2022"
campaign["day"] = campaign["day"].astype(str)
campaign["last_contact_date"] = campaign["year"] + "-" + campaign["month"] + "-" + campaign["day"]
campaign

,client_id,number_contacts,month,day,contact_duration,pdays,previous_campaign_contacts,previous_outcome,campaign_outcome,campaign_id,year,last_contact_date
0,0,1,May,13,261,999,0,NaN,0,1,2022,2022-May-13
1,1,1,May,19,149,999,0,NaN,0,1,2022,2022-May-19
2,2,1,May,23,226,999,0,NaN,0,1,2022,2022-May-23
3,3,1,May,27,151,999,0,NaN,0,1,2022,2022-May-27
4,4,1,May,3,307,999,0,NaN,0,1,2022,2022-May-3
...,...,...,...,...,...,...,...,...,...,...,...,...
41183,41183,1,Nov,30,334,999,0,NaN,1,1,2022,2022-Nov-30
41184,41184,1,Nov,6,383,999,0,NaN,0,1,2022,2022-Nov-6
41185,41185,2,Nov,24,189,999,0,NaN,0,1,2022,2022-Nov-24
41186,41186,1,Nov,17,442,999,0,NaN,1,1,2022,2022-Nov-17


In [62]:
campaign.drop(columns=["month", "day", "year"], inplace=True)

In [63]:
campaign

,client_id,number_contacts,contact_duration,pdays,previous_campaign_contacts,previous_outcome,campaign_outcome,campaign_id,last_contact_date
0,0,1,261,999,0,NaN,0,1,2022-May-13
1,1,1,149,999,0,NaN,0,1,2022-May-19
2,2,1,226,999,0,NaN,0,1,2022-May-23
3,3,1,151,999,0,NaN,0,1,2022-May-27
4,4,1,307,999,0,NaN,0,1,2022-May-3
...,...,...,...,...,...,...,...,...,...
41183,41183,1,334,999,0,NaN,1,1,2022-Nov-30
41184,41184,1,383,999,0,NaN,0,1,2022-Nov-6
41185,41185,2,189,999,0,NaN,0,1,2022-Nov-24
41186,41186,1,442,999,0,NaN,1,1,2022-Nov-17


In [65]:
campaign["last_contact_date"] = pd.to_datetime(campaign["last_contact_date"], format="%Y-%b-%d")
campaign

,client_id,number_contacts,contact_duration,pdays,previous_campaign_contacts,previous_outcome,campaign_outcome,campaign_id,last_contact_date
0,0,1,261,999,0,NaN,0,1,2022-05-13
1,1,1,149,999,0,NaN,0,1,2022-05-19
2,2,1,226,999,0,NaN,0,1,2022-05-23
3,3,1,151,999,0,NaN,0,1,2022-05-27
4,4,1,307,999,0,NaN,0,1,2022-05-03
...,...,...,...,...,...,...,...,...,...
41183,41183,1,334,999,0,NaN,1,1,2022-11-30
41184,41184,1,383,999,0,NaN,0,1,2022-11-06
41185,41185,2,189,999,0,NaN,0,1,2022-11-24
41186,41186,1,442,999,0,NaN,1,1,2022-11-17


In [66]:
client.to_csv("client.csv", index=False)
campaign.to_csv("campaign.csv", index=False)
economics.to_csv("economics.csv", index=False)

In [67]:
client_table = """CREATE TABLE client
(
    id SERIAL PRIMARY KEY,
    age INTEGER,
    job TEXT,
    marital TEXT,
    education TEXT,
    credit_default BOOLEAN,
    housing BOOLEAN,
    loan BOOLEAN
);
\copy client from 'client.csv' DELIMITER ',' CSV HEADER
"""

campaign_table = """CREATE TABLE campaign
(
    campaign_id SERIAL PRIMARY KEY,
    client_id SERIAL references client (id),
    number_contacts INTEGER,
    contact_duration INTEGER,
    pdays INTEGER,
    previous_campaign_contacts INTEGER,
    previous_outcome BOOLEAN,
    campaign_outcome BOOLEAN,
    last_contact_date DATE    
);
\copy campaign from 'campaign.csv' DELIMITER ',' CSV HEADER
"""

economics_table = """CREATE TABLE economics
(
    client_id SERIAL references client (id),
    emp_var_rate FLOAT,
    cons_price_idx FLOAT,
    euribor_three_months FLOAT,
    number_employed FLOAT
);
\copy economics from 'economics.csv' DELIMITER ',' CSV HEADER
"""